In [1]:
!pip install scikit-optimize

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

import math
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
filepath = '../input/bt4222-project/modelling_dataset.csv'
df = pd.read_csv(filepath)
df

,Area (SQM),Unit Price ($ PSM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
0,95.0,6316.0,1.0,124.3,33,705.752731,1207.822015,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,138.0,13833.0,1.0,124.3,58,1233.947139,768.529003,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,99.0,8990.0,10.0,124.3,50,1039.586179,816.818037,80.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,111.0,6306.0,6.0,124.3,33,509.516515,501.364218,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,122.0,13934.0,10.0,124.3,58,1253.733260,554.491114,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54669,192.0,8229.0,4.0,153.3,66,526.868822,464.021930,76.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
54670,111.0,7838.0,14.0,153.3,66,535.286185,490.669867,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
54671,142.0,13028.0,14.0,153.3,133,2075.609094,600.145285,72.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
54672,58.0,13931.0,6.0,153.3,51,397.651025,539.116552,91.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [4]:
## split dataset
X = df.drop(columns = ['Unit Price ($ PSM)'])
y = df['Unit Price ($ PSM)']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    shuffle=True, 
                                                    random_state=42)

X_train

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
11187,129.0,9.0,139.2,66,313.319878,1514.605671,85.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
18593,66.0,13.0,142.3,63,408.305136,1517.388799,84.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27013,61.0,58.0,137.7,85,1529.338980,148.651892,86.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0
39116,105.0,18.0,141.6,66,270.671774,225.750354,86.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
6042,111.0,37.0,132.6,60,339.216214,258.201238,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44732,107.0,3.0,145.2,36,102.333066,622.895879,73.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54343,66.0,2.0,153.3,64,414.799433,805.354104,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
38158,74.0,3.0,141.6,142,774.750823,466.130105,76.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
860,91.0,14.0,124.3,133,658.187930,310.779399,83.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [5]:
def all_transform(X_train, X_test):
    all_features = list(X_train.columns)

    standardScale_vars = ['Area (SQM)',
                          'Floor Number',
                          'PPI',
                          'Average Cases Per Year',
                          'Nearest Primary School',
                          'nearest_station_distance']

    minMax_vars = ['Remaining Lease']

    remaining_features = [x for x in all_features if x not in standardScale_vars and x not in minMax_vars]

    s_scaler = StandardScaler()
    mm_scaler = MinMaxScaler()

    s_scaled = pd.DataFrame(s_scaler.fit_transform(X_train.loc[:, standardScale_vars].copy()), columns=standardScale_vars, index=X_train.index)
    mm_scaled = pd.DataFrame(mm_scaler.fit_transform(X_train.loc[:, minMax_vars].copy()), columns=minMax_vars, index=X_train.index)

    X_train = pd.concat([s_scaled, 
                         mm_scaled, 
                         X_train.loc[:, remaining_features].copy()], axis=1)

    s_scaled = pd.DataFrame(s_scaler.transform(X_test.loc[:, standardScale_vars].copy()), columns=standardScale_vars, index=X_test.index)
    mm_scaled = pd.DataFrame(mm_scaler.transform(X_test.loc[:, minMax_vars].copy()), columns=minMax_vars, index=X_test.index)

    X_test = pd.concat([s_scaled, 
                        mm_scaled, 
                        X_test.loc[:, remaining_features].copy()], axis=1)
    
    return X_train, X_test

In [6]:
X_train, X_test = all_transform(X_train, X_test)
X_train

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
11187,0.291530,-0.109176,-0.236229,0.243812,-0.771624,1.226898,0.745098,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
18593,-1.125873,0.371998,0.210453,0.125051,-0.570640,1.231468,0.725490,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27013,-1.238366,5.785202,-0.452365,0.995966,1.801408,-1.015899,0.764706,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0
39116,-0.248433,0.973465,0.109589,0.243812,-0.861865,-0.889309,0.764706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
6042,-0.113442,3.259040,-1.187229,0.006290,-0.716828,-0.836027,0.784314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44732,-0.203436,-0.830937,0.628316,-0.943799,-1.218061,-0.237225,0.509804,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54343,-1.125873,-0.951230,1.795452,0.164638,-0.556898,0.062359,0.784314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
38158,-0.945886,-0.830937,0.109589,3.252428,0.204740,-0.494623,0.568627,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
860,-0.563412,0.492291,-2.383183,2.896144,-0.041901,-0.749697,0.705882,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [7]:
X_test

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
26109,0.134041,0.011118,-0.149775,0.243812,-0.629690,-0.453873,0.666667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
51446,-0.765898,-0.830937,1.147043,-0.864625,-1.058519,-0.912692,0.823529,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0
21609,0.224035,-0.349763,1.031770,2.896144,2.509617,0.157013,0.705882,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6390,-0.045947,-0.951230,-1.187229,-0.033297,0.062520,-0.976248,0.843137,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
40798,5.983642,3.018453,0.757998,-1.102147,0.138606,-0.501497,0.862745,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7350,1.056478,0.612585,-0.841411,-0.389580,0.018348,0.334854,0.509804,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24268,-0.248433,0.011118,0.440998,-0.349993,0.014873,1.248442,0.627451,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
29622,0.089044,-0.470056,-0.855820,0.402161,-0.675015,1.227770,0.647059,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
50030,-1.575843,0.492291,1.132634,0.243812,-0.740679,-0.669763,0.882353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(38271, 52)
(38271,)
(16403, 52)
(16403,)


In [9]:
df_error_log = []

## Linear Regression
### Randomised Search

In [10]:
regressor = LinearRegression()

# define parameters
param_grid = dict()
param_grid['fit_intercept'] = [True, False]

In [11]:
%%time
random_grid = RandomizedSearchCV(regressor, 
                                 param_grid, 
                                 n_iter=300, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1, 
                                 verbose = 1,
                                 cv=5,
                                 random_state = 42)
# execute search
random_grid_result = random_grid.fit(X_train, y_train)

# summarize result
print('Best Score: ', random_grid_result.best_score_)
print('Best Params: ', random_grid_result.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 2 is smaller than n_iter=300. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Best Score:  -1935.1503025695133
Best Params:  {'fit_intercept': True}
CPU times: user 279 ms, sys: 126 ms, total: 405 ms
Wall time: 2.86 s


In [12]:
y_try = random_grid_result.best_estimator_.predict(X_test)
math.sqrt(mean_squared_error(y_test, y_try))

1917.569053164017

In [13]:
y_train_pred = random_grid.predict(X_train)
y_test_pred = random_grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6061234882848
Test RMSE : 1917.569053164017
Train MAPE : 0.11931915382253643
Test MAPE : 0.11845210830488977


In [14]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': random_grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23918.286534
1,River Valley,14896.672074
2,Newton,11940.768636
3,Downtown Core,10819.091431
4,Southern Islands,9637.237580
5,Tanglin,9231.317383
6,Remaining Lease,9220.678859
7,Museum,8759.694645
8,Singapore River,7415.180480
9,Novena,6363.911784


In [15]:
error_log = {
    'model_type': 'linear',
    'search_type': 'randomized',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

## Ridge Regression
### Randomized Search

In [16]:
regressor = Ridge()

# define parameters
param_grid = dict()
param_grid['alpha'] = list(np.arange(0, 10,0.01))
param_grid['fit_intercept'] = [True, False]

In [17]:
%%time
random_grid = RandomizedSearchCV(regressor, 
                                 param_grid, 
                                 n_iter=300, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1, 
                                 verbose = 1,
                                 cv=5,
                                 random_state = 42)
# execute search
random_grid_result = random_grid.fit(X_train, y_train)

# summarize result
print('Best Score: ', random_grid_result.best_score_)
print('Best Params: ', random_grid_result.best_params_)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Score:  -1935.1238897507733
Best Params:  {'fit_intercept': True, 'alpha': 0.16}
CPU times: user 4.61 s, sys: 448 ms, total: 5.06 s
Wall time: 22.3 s


In [18]:
y_train_pred = random_grid.predict(X_train)
y_test_pred = random_grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6178152651964
Test RMSE : 1917.5658345990382
Train MAPE : 0.11933115994010582
Test MAPE : 0.11846870664129865


In [19]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': random_grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23770.265460
1,River Valley,14839.012443
2,Newton,11891.792980
3,Downtown Core,10764.982692
4,Southern Islands,9583.363581
5,Remaining Lease,9217.893492
6,Tanglin,9183.512781
7,Museum,8609.481753
8,Singapore River,7348.167925
9,Novena,6332.472504


In [20]:
error_log = {
    'model_type': 'ridge',
    'search_type': 'randomized',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

### Grid Search

In [21]:
%%time
grid = GridSearchCV(estimator=regressor, 
                    param_grid=param_grid, 
                    scoring='neg_root_mean_squared_error', 
                    verbose=1, 
                    n_jobs=-1, 
                    cv=5)

grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits
Best Score:  -1935.123226413801
Best Params:  {'alpha': 0.19, 'fit_intercept': True}
CPU times: user 29.9 s, sys: 2.72 s, total: 32.6 s
Wall time: 2min 29s


In [22]:
# overfit, 30% test
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6225504964734
Test RMSE : 1917.5675369445366
Train MAPE : 0.11933380695063688
Test MAPE : 0.11847207024657062


In [23]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23742.758108
1,River Valley,14828.318004
2,Newton,11882.705158
3,Downtown Core,10754.960892
4,Southern Islands,9573.380880
5,Remaining Lease,9217.372755
6,Tanglin,9174.656262
7,Museum,8581.786653
8,Singapore River,7335.751507
9,Novena,6326.654514


In [24]:
error_log = {
    'model_type': 'ridge',
    'search_type': 'grid',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

## Lasso
### Randomized search

In [25]:
regressor = Lasso()

# define parameters
param_grid = dict()
param_grid['alpha'] = list(np.arange(0, 10,0.01))
param_grid['fit_intercept'] = [True, False]
param_grid['warm_start'] = [True, False] # reuse the solution of the previous call to fit as initialization, otherwise, just erase the previous solution.

In [26]:
%%time
random_grid = RandomizedSearchCV(regressor, 
                                 param_grid, 
                                 n_iter=300, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1, 
                                 cv=5,
                                 random_state = 42)
# execute search
random_grid_result = random_grid.fit(X_train, y_train)

# summarize result
print('Best Score: ', random_grid_result.best_score_)
print('Best Params: ', random_grid_result.best_params_)

Best Score:  -1935.1466403883073
Best Params:  {'warm_start': True, 'fit_intercept': True, 'alpha': 0.02}
CPU times: user 15.3 s, sys: 2.61 s, total: 17.9 s
Wall time: 1min 51s


In [27]:
#30%
y_train_pred = random_grid.predict(X_train)
y_test_pred = random_grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6115046351806
Test RMSE : 1917.597742684988
Train MAPE : 0.11932505621883975
Test MAPE : 0.11845999915539777


In [28]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': random_grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23863.205321
1,River Valley,14861.157990
2,Newton,11909.324052
3,Downtown Core,10778.711476
4,Southern Islands,9596.476111
5,Remaining Lease,9218.578516
6,Tanglin,9195.085370
7,Museum,8668.858300
8,Singapore River,7367.684192
9,Novena,6338.400188


In [29]:
error_log = {
    'model_type': 'lasso',
    'search_type': 'randomized',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

### Grid Search

In [30]:
%%time
grid = GridSearchCV(estimator=regressor, 
                    param_grid=param_grid, 
                    scoring='neg_root_mean_squared_error', 
                    verbose=1, 
                    n_jobs=-1, 
                    cv=5)

grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 4000 candidates, totalling 20000 fits
Best Score:  -1935.1466403883073
Best Params:  {'alpha': 0.02, 'fit_intercept': True, 'warm_start': True}
CPU times: user 2min 58s, sys: 17.9 s, total: 3min 15s
Wall time: 25min 8s


In [31]:
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6115046351806
Test RMSE : 1917.597742684988
Train MAPE : 0.11932505621883975
Test MAPE : 0.11845999915539777


In [32]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23863.205321
1,River Valley,14861.157990
2,Newton,11909.324052
3,Downtown Core,10778.711476
4,Southern Islands,9596.476111
5,Remaining Lease,9218.578516
6,Tanglin,9195.085370
7,Museum,8668.858300
8,Singapore River,7367.684192
9,Novena,6338.400188


In [33]:
error_log = {
    'model_type': 'lasso',
    'search_type': 'grid',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

## Elastic Net
### Randomized Search

In [34]:
regressor = ElasticNet()

# define parameters
param_grid = dict()
param_grid['alpha'] = list(np.arange(0, 10,0.1))
param_grid['l1_ratio'] = list(np.arange(0, 1,0.1))
param_grid['fit_intercept'] = [True, False]
param_grid['warm_start'] = [True, False] # reuse the solution of the previous call to fit as initialization, otherwise, just erase the previous solution.

In [35]:
%%time
random_grid = RandomizedSearchCV(regressor, 
                                 param_grid, 
                                 n_iter=300, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1, 
                                 verbose=1,
                                 cv=5,
                                 random_state = 42)
# execute search
random_grid_result = random_grid.fit(X_train, y_train)

print('done')

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:880: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)


done
CPU times: user 19.7 s, sys: 3.33 s, total: 23 s
Wall time: 4min 17s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 71322601097.5075, tolerance: 76431228.46262392
  positive)


In [36]:
# 300 iterations
# summarize result
print('Best Score: ', random_grid_result.best_score_)
print('Best Params: ', random_grid_result.best_params_)

y_train_pred = random_grid.predict(X_train)
y_test_pred = random_grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Best Score:  -1935.1503025695129
Best Params:  {'warm_start': True, 'l1_ratio': 0.7000000000000001, 'fit_intercept': True, 'alpha': 0.0}
Train RMSE : 1930.6061234882848
Test RMSE : 1917.569053164017
Train MAPE : 0.11931915382253637
Test MAPE : 0.11845210830488971


In [37]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': random_grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23918.286534
1,River Valley,14896.672074
2,Newton,11940.768636
3,Downtown Core,10819.091431
4,Southern Islands,9637.237580
5,Tanglin,9231.317383
6,Remaining Lease,9220.678859
7,Museum,8759.694645
8,Singapore River,7415.180480
9,Novena,6363.911784


In [38]:
error_log = {
    'model_type': 'elasticnet',
    'search_type': 'randomized',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

### GridSearch

In [39]:
%%time
grid = GridSearchCV(estimator=regressor, 
                    param_grid=param_grid, 
                    scoring='neg_root_mean_squared_error', 
                    verbose=1, 
                    n_jobs=-1, 
                    cv=5)

grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 4000 candidates, totalling 20000 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:880: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)


Best Score:  -1935.1503025695129
Best Params:  {'alpha': 0.0, 'fit_intercept': True, 'l1_ratio': 0.0, 'warm_start': True}
CPU times: user 3min 1s, sys: 19.1 s, total: 3min 20s
Wall time: 57min 35s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 71322601097.5075, tolerance: 76431228.46262392
  positive)


In [40]:
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

Train RMSE : 1930.6061234882848
Test RMSE : 1917.569053164017
Train MAPE : 0.11931915382253637
Test MAPE : 0.11845210830488971


In [41]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': grid.best_estimator_.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23918.286534
1,River Valley,14896.672074
2,Newton,11940.768636
3,Downtown Core,10819.091431
4,Southern Islands,9637.237580
5,Tanglin,9231.317383
6,Remaining Lease,9220.678859
7,Museum,8759.694645
8,Singapore River,7415.180480
9,Novena,6363.911784


In [42]:
error_log = {
    'model_type': 'elasticnet',
    'search_type': 'grid',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

### Bayesian

In [43]:
%%time
from skopt import gp_minimize
from skopt.utils import use_named_args
from sklearn.model_selection import cross_val_score
from skopt.space import Real, Categorical

search_space = [Real(low=0, high=10, name='alpha'), 
                Real(low=0, high=1, name='l1_ratio'),
                Categorical(categories=[True,False], name='fit_intercept'),
                Categorical(categories=[True,False], name='warm_start'),
               ]

model = ElasticNet()

@use_named_args(search_space)
def evaluate_model(**params):
    model.set_params(**params)
    
    result = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
    estimate = -np.mean(result)
    
    return estimate

rmse_result = gp_minimize(evaluate_model, search_space)

print('Best RMSE: %.2f' % rmse_result.fun)
print('Best Parameters: alpha=%.2f, l1_ratio=%.2f, fit_intercept=%s, warm_start=%s' % (rmse_result.x[0], rmse_result.x[1], rmse_result.x[2], rmse_result.x[3]))

/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/conda/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/conda/lib/python3.7/site-packa

Best RMSE: 1935.15
Best Parameters: alpha=0.00, l1_ratio=1.00, fit_intercept=True, warm_start=False
CPU times: user 3min 34s, sys: 2min 34s, total: 6min 9s
Wall time: 13min 42s


In [44]:
Enet = ElasticNet(alpha=rmse_result.x[0], l1_ratio=rmse_result.x[1], fit_intercept=rmse_result.x[2], warm_start=rmse_result.x[3])
model = Enet.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_RMSE = math.sqrt(mean_squared_error(y_train, y_train_pred))
test_RMSE = math.sqrt(mean_squared_error(y_test, y_test_pred))

print('Train RMSE : ' + str(train_RMSE))
print('Test RMSE : ' + str(test_RMSE))

print('Train MAPE : ' + str(mean_absolute_percentage_error(y_train, y_train_pred)))
print('Test MAPE : ' + str(mean_absolute_percentage_error(y_test, y_test_pred)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)


Train RMSE : 1930.6061234882848
Test RMSE : 1917.569053164017
Train MAPE : 0.11931915382253637
Test MAPE : 0.11845210830488971


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 71322601097.5075, tolerance: 76431228.46262392
  positive)


In [45]:
feature_impt = pd.DataFrame({'feature': list(X.columns),
                             'coefficient': Enet.coef_})
feature_impt.sort_values(by='coefficient', ascending=False, inplace=True)
feature_impt.reset_index(drop=True, inplace=True)

feature_impt

,feature,coefficient
0,Orchard,23918.286534
1,River Valley,14896.672074
2,Newton,11940.768636
3,Downtown Core,10819.091431
4,Southern Islands,9637.237580
5,Tanglin,9231.317383
6,Remaining Lease,9220.678859
7,Museum,8759.694645
8,Singapore River,7415.180480
9,Novena,6363.911784


In [46]:
error_log = {
    'model_type': 'elasticnet',
    'search_type': 'bayesian',
    'train_rmse': train_RMSE,
    'test_rmse': test_RMSE, 
    'train_mape': mean_absolute_percentage_error(y_train, y_train_pred),
    'test_mape': mean_absolute_percentage_error(y_test, y_test_pred),
    'top_features': feature_impt.to_dict('records')
}

df_error_log.append(error_log)

In [47]:
df_error_log = pd.DataFrame(df_error_log)
df_error_log

,model_type,search_type,train_rmse,test_rmse,train_mape,test_mape,top_features
0,linear,randomized,1930.606123,1917.569053,0.119319,0.118452,"[{'feature': 'Orchard', 'coefficient': 23918.2..."
1,ridge,randomized,1930.617815,1917.565835,0.119331,0.118469,"[{'feature': 'Orchard', 'coefficient': 23770.2..."
2,ridge,grid,1930.622550,1917.567537,0.119334,0.118472,"[{'feature': 'Orchard', 'coefficient': 23742.7..."
3,lasso,randomized,1930.611505,1917.597743,0.119325,0.118460,"[{'feature': 'Orchard', 'coefficient': 23863.2..."
4,lasso,grid,1930.611505,1917.597743,0.119325,0.118460,"[{'feature': 'Orchard', 'coefficient': 23863.2..."
5,elasticnet,randomized,1930.606123,1917.569053,0.119319,0.118452,"[{'feature': 'Orchard', 'coefficient': 23918.2..."
6,elasticnet,grid,1930.606123,1917.569053,0.119319,0.118452,"[{'feature': 'Orchard', 'coefficient': 23918.2..."
7,elasticnet,bayesian,1930.606123,1917.569053,0.119319,0.118452,"[{'feature': 'Orchard', 'coefficient': 23918.2..."


In [48]:
df_error_log.to_csv('linear_regularized_error_log.csv', index=False)